In [20]:
import neo4j

host = "localhost"
username = "neo4j"
password = "1234qwer"

driver = neo4j.GraphDatabase.driver(f'bolt://localhost:7687', auth=(username, password))
session = driver.session()

def create_graph(query):
    session.run(query)

In [18]:
import psycopg2

pg_host = "localhost"
pg_database = "demo"
pg_user = "postgres"
pg_password = "postgres"

# Establish PostgreSQL connection
pg_conn = psycopg2.connect(
    host=pg_host,
    database=pg_database,
    user=pg_user,
    password=pg_password
)
pg_cursor = pg_conn.cursor()

In [34]:
session.run("""
MATCH (source:Requirement)-[:CONTAINS]->(target:CORE|VERB|USER|PLATFORM|HARDWARE)
RETURN gds.graph.project(
  'myGraph011',
  source,
  target,
  {
    sourceNodeProperties: {},
    targetNodeProperties: target { .bert_embedding }
  },
  { undirectedRelationshipTypes: ['*'] }
)
""")

In [35]:
session.run("""
CALL gds.fastRP.write('myGraph011',
  {
    embeddingDimension: 768,
    writeProperty: 'fastrp-bert-embedding'
  }
)
YIELD nodePropertiesWritten
""")

In [36]:
session.run("""
MATCH (p:Requirement)
RETURN gds.graph.project(
  'requirementGraph',
  p,
  null,
  {
    sourceNodeProperties: p { .`fastrp-bert-embedding` },
    targetNodeProperties: {}
  }
)
""")

In [38]:
records, summary, key = driver.execute_query("""
CALL gds.knn.stream('requirementGraph', {
    topK: 1,
    nodeProperties: { `fastrp-bert-embedding`: 'EUCLIDEAN'},
    // The following parameters are set to produce a deterministic result
    randomSeed: 1337,
    concurrency: 1,
    sampleRate: 1.0,
    deltaThreshold: 0.0
})
YIELD node1, node2, similarity                                        
RETURN gds.util.asNode(node1).id AS id1, gds.util.asNode(node2).id AS id2, similarity
ORDER BY similarity DESC
""")

for record in records:
    pg_cursor.execute("INSERT INTO result_graph (source_id, target_id, similarity, method, embedding) VALUES (%s, %s, %s, %s, %s)", (record[0], record[1], record[2], 'EUCLIDEAN', 'BERT'))
    pg_conn.commit()

: 